In [1]:
import socket
import threading
import time
from typing import Optional
from io import StringIO
import contextlib
import sys
import traceback

class StreamingOutput:
    def __init__(self, callback):
        self.callback = callback
        self.buffer = []

    def write(self, text):
        self.callback(text)
        self.buffer.append(text)
        
    def flush(self):
        pass

    def getvalue(self):
        return ''.join(self.buffer)
        
class UDPPythonServer:
    def __init__(self, port: int = 12345):
        self.port = port
        self.running = False
        self.server_thread: Optional[threading.Thread] = None
        self.sock: Optional[socket.socket] = None
        self.namespace = {}
        self.current_client = None
        
    def _create_socket(self):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        if hasattr(socket, 'SO_REUSEPORT'):
            self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEPORT, 1)
        self.sock.bind(('127.0.0.1', self.port))
        self.sock.settimeout(0.5)

    def _send_output(self, text):
        if self.current_client and self.running:
            self.sock.sendto(text.encode('utf-8'), self.current_client)
            
    def _execute_code(self, code: str) -> str:
        output = StreamingOutput(self._send_output)
        error_output = StreamingOutput(self._send_output)
        
        with contextlib.redirect_stdout(output), contextlib.redirect_stderr(error_output):
            try:
                exec(code, self.namespace)
            except Exception:
                traceback.print_exc(file=error_output)
                
        return output.getvalue() or "Code executed successfully"
            
    def _handle_connection(self):
        while self.running:
            try:
                data, addr = self.sock.recvfrom(4096)
                if addr[0] != '127.0.0.1':
                    continue
                
                self.current_client = addr
                code = data.decode('utf-8')
                self._execute_code(code)
                self.current_client = None
                    
            except socket.timeout:
                continue
            except Exception as e:
                if self.running:
                    print(f"Server error: {str(e)}")
                self.current_client = None
                    
    def start(self):
        if self.running:
            print("Server is already running")
            return
            
        try:
            self.running = True
            self._create_socket()
            self.server_thread = threading.Thread(target=self._handle_connection)
            self.server_thread.start()
            print(f"Server started on port {self.port}")
        except Exception as e:
            self.running = False
            if self.sock:
                self.sock.close()
                self.sock = None
            raise e
            
    def stop(self):
        if not self.running:
            print("Server is not running")
            return
            
        self.running = False
        if self.server_thread:
            self.server_thread.join()
        if self.sock:
            self.sock.close()
            self.sock = None
        self.namespace.clear()
        print("Server stopped")

In [2]:
def send_code(code: str, port: int = 12345):
    client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    client.settimeout(0.1)
    client.sendto(code.encode('utf-8'), ('127.0.0.1', port))
    
    # Keep receiving until 2 seconds of no new data
    last_receive_time = time.time()
    while True:
        try:
            data, _ = client.recvfrom(4096)
            print(data.decode('utf-8'), end='', flush=True)
            last_receive_time = time.time()
        except socket.timeout:
            # If no data for 2 seconds, assume we're done
            if time.time() - last_receive_time > 2.0:
                break
            continue
        except Exception as e:
            print(f"Error receiving: {e}")
            break
            
    client.close()

In [3]:
server = UDPPythonServer(port=12345)
server.start()

Server started on port 12345


In [4]:
code="""
import time
for i in range(10):
    print(i)
    time.sleep(1)
"""

In [5]:
send_code(code)

Error receiving: 'StreamingOutput' object has no attribute 'callback'


In [17]:
server.stop()

Server stopped
